# 안정성

코드는 안전하고 예측 가능해야 합니다. 잠재적인 오류를 최소화하고 안정성을 높이기 위해 적절한 예외 처리와 메모리 관리를 고려해야 합니다.


## 예외 처리와 오류

### 나쁜 예:
```cpp
#include <iostream>
#include <stdexcept>
#include <unistd.h>
#include <arpa/inet.h>
#include <sys/socket.h>
using namespace std;

int main() {
    int sockfd = socket(AF_INET, SOCK_STREAM, 0);
    if (sockfd < 0) {
        cerr << "Error: Failed to create socket." << endl;
        return 1;
    }

    struct sockaddr_in serverAddr;
    serverAddr.sin_family = AF_INET;
    serverAddr.sin_port = htons(8080);
    serverAddr.sin_addr.s_addr = inet_addr("127.0.0.1");

    if (connect(sockfd, (struct sockaddr*)&serverAddr, sizeof(serverAddr)) < 0) {
        cerr << "Error: Failed to connect to server." << endl;
        close(sockfd);
        return 1;
    }

    // 서버와의 통신을 위한 작업 수행
    // ...

    close(sockfd);
    return 0;
}

```

위의 코드는 네트워크 소켓을 생성하고 서버에 연결하는 작업을 수행합니다. 그러나 네트워크 오류가 발생할 경우에 대한 예외 처리가 부족합니다.

### 좋은 예:
```cpp
#include <iostream>
#include <stdexcept>
#include <unistd.h>
#include <arpa/inet.h>
#include <sys/socket.h>
using namespace std;

int main() {
    try {
        int sockfd = socket(AF_INET, SOCK_STREAM, 0);
        if (sockfd < 0) {
            throw runtime_error("Failed to create socket.");
        }

        struct sockaddr_in serverAddr;
        serverAddr.sin_family = AF_INET;
        serverAddr.sin_port = htons(8080);
        serverAddr.sin_addr.s_addr = inet_addr("127.0.0.1");

        if (connect(sockfd, (struct sockaddr*)&serverAddr, sizeof(serverAddr)) < 0) {
            throw runtime_error("Failed to connect to server.");
        }

        // 서버와의 통신을 위한 작업 수행
        // ...

        close(sockfd);
        return 0;
    } catch (const runtime_error& e) {
        cerr << "Error: " << e.what() << endl;
        return 1;
    }
}

```
이제 네트워크 통신 도중 오류가 발생할 경우 예외를 던져서 적절히 처리합니다. 예외가 발생하면 오류 메시지를 출력하고 프로그램을 종료합니다. 이렇게 함으로써 프로그램이 네트워크 오류에 대응하여 안정적으로 동작할 수 있게 됩니다.


## 메모리 관리

### 나쁜 예:
```cpp
#include <iostream>
using namespace std;

int* createIntArray(int size) {
    int arr[size]; // 스택에 배열을 생성하여 반환
    return arr;
}

int main() {
    int* arrPtr = createIntArray(5);
    // arrPtr을 사용하려고 하지만 스택에 할당된 배열은 함수가 종료되면 사라짐
    // arrPtr은 더 이상 유효한 메모리를 가리키지 않음
    return 0;
}
```
함수 내에서 스택에 할당된 배열을 반환하고 있으므로 함수가 종료되면 해당 배열은 사라지고 포인터는 더 이상 유요한 메모리를 가르키지 않습니다.

### 좋은 예:
```cpp
#include <iostream>
using namespace std;

int* createIntArray(int size) {
    int* arr = new int[size]; // 힙에 배열을 동적으로 할당하여 반환
    return arr;
}

int main() {
    int* arrPtr = createIntArray(5);
    // arrPtr을 안전하게 사용할 수 있음
    delete[] arrPtr; // 메모리 누수를 방지하기 위해 메모리를 해제해야 함
    return 0;
}
```

동적으로 메모리를 할당하여 반환하고, 사용 후에는 메모리를 명시적으로 해제하여 메모리 누수를 방지합니다. 

이를 통해 프로그램의 안정성을 높일 수 있습니다.